# Interpolation Functions for Circumbinary Planet Stability


The go-to source for determining whether a circumbinary planet or CBP (i.e., planet that two stars completely) has been an empirically determined fitting formula by Holman & Wiegert (1999).  In their study, the key assumptions are that the planet begins on a circular, coplanar orbit and does not have sufficient mass to influence the orbits of the massive binary stars (i.e., massless test particle).  Through the discoveries made by the Kepler Space Telescope, these assumptions have not held up to reality.  It is true that we may expect CBPs to begin with on nearly circular and coplanar orbits relative to the host binary, but disk interactions or planet-planet interactions over the lifetime of the system can perturb the discovered planets away from our ideal assumption.  Secondly, the parameter estimation of CBPs (e.g., planetary mass and eccentricity) **depend** on how the planet perturbs the binary orbit enough to affect the eclipse times.  

Quarles et al. (2018) provides a somewhat different approach.  Instead of using a fitting function based on some physical parameters (e.g., stellar dynamical mass ratio $\mu$ and binary eccentricity $e_{bin}$), they use an interpolation scheme that is built upon a much larger set of N-body simulations.  In these simulations, all three bodies interact with on another gravitationally and more accurately represent what we expect to observe.  This Jupyter notebook will show you how to utilize the results of Quarles et al. (2018) using Kepler-16 as an example.

# Physical observables
For eclipsing binary systems, one can usually obtain an estimate of the mass quotient $q=M_B/M_A$, binary orbital period $P_{bin}$, and the binary eccentricity $e_{bin}$ from modeling the observed photometric data (i.e., light curve) and radial velocity measurements (RV curve) together.  More intense modeling can help determine the individual masses of the stars, but we can convert between the observational mass quotient $q$ to the dynamical mass ratio $\mu$ through the following:

$\mu = \frac{M_B}{M_A+M_B} = \frac{q}{1+q}$

or

$q = \frac{\mu}{1-\mu}$

# Kepler-16
From Doyle et al. (2011) and Bender et al. (2012), we have measurements of the Kepler-16 binary for $q=0.2994 \pm 0.0031$, $e_{bin} = 0.15894 \pm 0.00079$, and $P_{bin} = 41.077580 \pm 0.000008$ days.  Now to the preliminaries of the code:

In [3]:
import numpy as np  #We need numpy for the convenience function genfromtxt
from scipy.interpolate import griddata,interp2d  #We need this to map our data to a grid and interpolate between grid points

def get_stability_limit(f,mu,e_bin,P_bin):
    return f(mu,e_bin)[0]**(1.5)*P_bin

The import statements above include the standard packages from numpy and scipy.  The function **get_stability_limit** is something for convenience so that we may generalize later and is passed a set of parameters.  The interpolation function $f$ that we create using scipy and the binary parameters.  Next, let's create the interpolation function.

In [4]:
data = np.genfromtxt("a_crit.txt",delimiter=',',comments='#')  #The data contained in this repository

X = data[:,0] #mu
Y = data[:,1] #e_bin
Z = data[:,2] #a_c/a_bin (this ratio is being converted to period in our stability fuction)

xi = np.linspace(0,0.5,51)
yi = np.linspace(0,0.8,81)
zi = griddata((X,Y),Z,(xi[None,:],yi[:,None]),method = 'linear',fill_value=0)  #make the grid

f = interp2d(xi, yi, zi, kind='linear') # make the 2d interpolation

In [10]:
mu_1 = 0.2937/(1 + 0.2937)  #converting from q --> mu  Doyle et al. (2011)
mu_2 = 0.2994/(1 + 0.2994)  #converting from q --> mu  Bender et al. (2012)
e_bin_1 = 0.15944
e_bin_2 = 0.15894
P_bin = 41.077580

print("Using Doyle, P_c = %3.4f days" % get_stability_limit(f,mu_1,e_bin_1,P_bin))
print("Using Bender, P_c = %3.4f days" % get_stability_limit(f,mu_2,e_bin_2,P_bin))

Using Doyle, P_c = 181.8849 days
Using Bender, P_c = 182.1355 days


Let's compare the critical period $P_c$ to the measured orbital period of the CBP $P_b = 228.776$ days.  The ratio of the CBP orbital period to the critical period (using Bender) is $P_b/P_c = 1.25607$, which tells us that Kepler-16b is ~25% beyond the critical period, or ~15% beyond the critical semimajor axis ratio $a_c/a_{bin}$.